## Uppsalakyrkogård kulturpersoner
version 0.1
* [This Notebook](https://github.com/salgo60/open-data-examples/blob/master/Uppsalakyrkogård%20kulturpersoner.ipynb)
* [Map](https://kulturpersoner.uppsalakyrkogardar.se/karta/) / [Lista](https://kulturpersoner.uppsalakyrkogardar.se/galleri/)
* [GIST](https://gist.github.com/salgo60/34e9a6bfc223687951ad827bf84b330d)